In [0]:
# imports
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
# parameters 
# Widget parameters
dbutils.widgets.text("checkpoint_path", "s3://databricks-storage-4052354327981619/checkpoints/silver_consolidated", "Checkpoint Path")
dbutils.widgets.text("catalog", "workspace_bank", "catalog")
dbutils.widgets.text("bronze_schema", "bronze", "Bronze Schema")
dbutils.widgets.text("silver_schema", "silver", "Silver Schema")
dbutils.widgets.dropdown("trigger_mode", "processingTime", ["processingTime", "availableNow"], "Trigger Mode")
dbutils.widgets.text("trigger_interval", "30 seconds", "Trigger Interval")

# Get parameters
checkpoint_path = dbutils.widgets.get("checkpoint_path").rstrip("/")
catalog = dbutils.widgets.get("catalog")
bronze_schema = dbutils.widgets.get("bronze_schema")
silver_schema = dbutils.widgets.get("silver_schema")
trigger_mode = dbutils.widgets.get("trigger_mode")
trigger_interval = dbutils.widgets.get("trigger_interval")

# Table names
bronze_transactions_table = f"{catalog}.{bronze_schema}.credit_card_transactions"
silver_merchants_table = f"{catalog}.{silver_schema}.merchants"
silver_target_table = f"{catalog}.{silver_schema}.consolidated_credit_card_merchant_trx"

print(f"📂 Checkpoint Path: {checkpoint_path}")
print(f"📊 Bronze Transactions: {bronze_transactions_table}")
print(f"📊 Silver Merchants: {silver_merchants_table}")
print(f"📊 Silver Target: {silver_target_table}")
print(f"⏱️  Trigger Mode: {trigger_mode}")

In [0]:
# Read bronze transactions as stream
df_bronze_stream = (
    spark.readStream
    .format("delta")
    .table(bronze_transactions_table)
)

print("✅ Bronze stream configured")
print(f"Schema: {df_bronze_stream.schema}")

In [0]:
# Read merchants as static (batch)
df_merchants = spark.read.table(silver_merchants_table)
print(f"📊 Merchants loaded: {df_merchants.count():,} records")

In [0]:
# Data Quality

# UDF for account_id validation
def validate_account_id(account_id):
    """Validate account_id is within expected range"""
    if account_id is None:
        return False
    if account_id < 1000000 or account_id > 10000000:
        return False
    return True

# Register UDFs
validate_account_id_udf = F.udf(validate_account_id, BooleanType())

# Add data quality flags
df_with_quality = (
    df_bronze_stream
    .withColumn("is_valid_account_id", 
                F.when((F.col("account_id") >= 1000000) & (F.col("account_id") <= 10000000), True)
                .otherwise(False))
    .withColumn("is_valid_transaction_amount",
                F.when((F.col("transaction_amount") > 0) & (F.col("transaction_amount") < 100000), True)
                .otherwise(False))
    .withColumn("is_valid_merchant_id",
                F.when(F.col("merchant_id").isNotNull(), True)
                .otherwise(False))
)

In [0]:
# Get the latest partition date from merchants table
latest_partition = (
    spark.read.table(silver_merchants_table)
    .select(F.max("partition_date").alias("max_date"))
    .collect()[0]["max_date"]
)

# Read only the latest partition
df_merchants = (
    spark.read.table(silver_merchants_table)
    .filter(F.col("partition_date") == latest_partition)
)

# Join with merchants (stream-static join)
df_with_merchants = (
    df_with_quality
    .join(
        df_merchants.select(
            F.col("merchant_id"),
            F.col("merchant_name"),
            F.col("merchant_region"),
            F.col("merchant_location"),
            F.col("merchant_region")
        ),
        "merchant_id",
        "left"  # Left join to keep transactions even if merchant not found
    )
)

In [0]:
# # deduplication

# df_deduplicated = (
#     df_with_campaign_status
#     .withWatermark("transaction_timestamp", "30 seconds")  # Handle late data
#     .dropDuplicates(["transaction_id", "merchant_category"])  # Remove duplicates by transaction_id and campaign_merchant_category
# )

In [0]:
# Add silver layer processing metadata

df_silver = (
    df_with_merchants
    .withColumn("_silver_processing_timestamp", F.current_timestamp())
    .withColumn("_silver_processing_date", F.current_date())
    .withColumn("_data_quality_score",
                F.when(F.col("is_valid_account_id") & 
                      F.col("is_valid_transaction_amount") & 
                      F.col("is_valid_merchant_id"), 100)
                .when(F.col("is_valid_account_id") & 
                      F.col("is_valid_transaction_amount"), 80)
                .when(F.col("is_valid_account_id"), 60)
                .otherwise(0))
)

In [0]:
df_final = df_silver.select(
    # Transaction core fields
    "transaction_id",
    "account_id",
    "transaction_amount",
    "transaction_timestamp",
    "transaction_status",
    
    # Merchant fields
    "merchant_id",
    "merchant_name",
    "merchant_category",
    "merchant_region",
    "merchant_location",
    
    # Data quality flags
    "is_valid_account_id",
    "is_valid_transaction_amount",
    "is_valid_merchant_id",
    "_data_quality_score",
    
    # Export metadata (from bronze)
    "export_id",
    "export_ts",
    "export_date",
    
    # Bronze metadata
    "_ingestion_timestamp",
    "_source_file",
    "_ingest_date",
    
    # Silver metadata
    "_silver_processing_timestamp",
    "_silver_processing_date"
)

print("✅ Final schema prepared")

In [0]:
# Configure trigger
def get_trigger():
    if trigger_mode == "availableNow":
        return {"availableNow": True}
    else:  # processingTime
        return {"processingTime": trigger_interval}

# Write to silver table
query = (
    df_final
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true")
    .trigger(**get_trigger())
    .toTable(silver_target_table)
)

print(f"✅ Streaming query started")
print(f"📊 Writing to: {silver_target_table}")
print(f"🔄 Query ID: {query.id}")
print(f"📍 Checkpoint: {checkpoint_path}")